# COMPSCI589 Homework 3
##### Chang Liu, 3.28.2022

## Programming: Random Forest Algorithm

In [58]:
import csv
from pprint import pprint
import numpy as np
import math
from sklearn.model_selection import train_test_split
from collections import Counter
import matplotlib.pyplot as plt


# load data
house_data = []

# import data, include encoding to ommit BOM  
with open('house_votes_84.csv', 'r', encoding='utf-8-sig') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        if len(row) != 0: # skip empty lines
            house_data.append(row)

house_attributes = house_data[0]
# drop attributes name from the data
house_data = np.array(house_data[1:]).astype(int)

In [59]:
# list_of_tag => entropy
def entropy(data):
    cnt = list(Counter(data).values())
    total = len(data)
    return sum([-k/total * math.log(k/total, 2) for k in cnt])

def gini(data):
    cnt = list(Counter(data).values())
    total = len(data)
    return 1 - sum([k/total * (k/total) for k in cnt])

# house_data(w/ tags)->entropy_of_all_attributes[]
def entropy_in_list(data, attr_list, algo):
    attr_count = len(data.T) - 1
    total_entry = len(data)
    ent_list = []
    # for each attributes, calculate information gain
    for i in range(attr_count):
        # skip if not in attr_list
        if(not i in attr_list):
            ent_list.append(1000) # infinate high entropy
            continue
        # three possible options are hard coded
        # organize data by options
        v_tag = {0: [], 1: [], 2: []}
        for j in range(len(data)):
            v_tag[data[j][i]].append(data[j][-1])
        # sum of entropy of each option
        ent = sum( len(v_tag[val])/total_entry * algo(v_tag[val]) for val in v_tag )
        ent_list.append(ent)
    return ent_list

In [67]:
class Node:
    def __init__(self, isLeaf=False, attr_index=None, tag=None):
        self.attr_index = attr_index
        self.isLeaf = isLeaf
        self.tag = tag
        self.possible_options = None
        self.option = None
        self.children = []

    def print_tree(self, indent=0):
        ind_str = ' ' * indent
        if self.isLeaf:
            print( ind_str + "Leaf:tag", self.tag, "vote", self.option)
        else:
            print( ind_str + "Node:", house_attributes[self.attr_index], "vote", self.option)
            for child in self.children:
                child.print_tree(indent+2)


def build_decision_tree(data, attr_list, algo):
    # If there are no more attributes that can be tested, return the most common tag
    if (not attr_list): # equivelent to len(attr_list) == 0
        return Node( isLeaf=True, tag=Counter(data.T[-1]).most_common()[0][0])

    # If there are no more data
    if (not data.any()):
        return Node( isLeaf=True, tag=None)

    tags = data.T[-1]
    original_entropy = entropy(tags)

    # If there is only one tag, return the tag
    if (original_entropy == 0):
        return Node( isLeaf=True, tag=Counter(tags).most_common()[0][0])
    # this subtraction is not necessary, just to be justify the name 'info_gain'
    if algo=='gini':
        gini_val = entropy_in_list(data, attr_list, gini)
        decision_attr = np.argmin(gini_val)
    elif algo=='entropy':
        info_gain = [original_entropy - ent for ent in entropy_in_list(data, attr_list, entropy)]
        decision_attr = np.argmax(info_gain)

    # get the index of the attribute with highest information gain
    nd = Node( isLeaf=False, attr_index=decision_attr) 
    # keep track of a majority tag in case the leaf is None
    nd.tag = Counter(data.T[-1]).most_common()[0][0]
    # remove decision attribute from attr_list
    new_attr_list = [x for x in attr_list if x != decision_attr]
    # since the option list for all attributes are the same
    # so this is hard coded

    nd.possible_options = set(data[decision_attr])
    # build subtree for each option 
    for v in nd.possible_options:
        # filtered data
        new_data = data[data.T[decision_attr] == v]
        subtree = build_decision_tree(new_data, new_attr_list, algo) 
        subtree.option = v
        nd.children.append(subtree)
    return nd

# temporary test data
temp_data = house_data[:5, -5:]
# temp_data = np.array([[0,0,1,2,2,2,1,0,0,2,0,1,1,2], [0,0,1,1,1,0,1,0,1,1,1,1,1,0]]).T
# print(temp_data)

In [61]:
def predict(data, node):
    for child in node.children:
        if data[node.attr_index] == child.option:
            if child.isLeaf:
                if child.tag == None:
                    return node.tag
                else:
                    return child.tag
            else:
                return predict(data, child)
    return None;

def evaluate(data, tree):
    correct = 0
    for i in range(len(data)):
        if data[i][-1] == predict(data[i], tree):
            correct += 1
    return correct / len(data)

def dispatch(data, algo, random_state=42):
    train, test = train_test_split(data, test_size=0.2, shuffle=True, random_state=random_state)
    tree = build_decision_tree(train, list(range(len(house_attributes)-1)), algo)
    tree.print_tree()
    return evaluate(train, tree), evaluate(test, tree)

In [68]:
eval_train, eval_test = dispatch(temp_data, "entropy")

print(eval_train, eval_test)